In [14]:
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from gensim.models import Word2Vec
############pyspark#############
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql import functions
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType
from pyspark import SparkContext
from pyspark.sql import SparkSession


In [15]:
def time2cov(value):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(value))

In [16]:
def map_hour(x):
    if (7<=x<=12):
        return 1
    elif (13 <= x<=20):
        return 2
    else:
        return 3

In [17]:
def slide_cnt(data):

    print('当前日期前一天的cnt')
    for d in range(19, 26):  # 18到24号
        df1 = data[data['day'] == d - 1]
        df2 = data[data['day'] == d]  # 19到25号
        user_cnt = df1.groupby(by='user_id').count()['instance_id'].to_dict()
        item_cnt = df1.groupby(by='item_id').count()['instance_id'].to_dict()
        shop_cnt = df1.groupby(by='shop_id').count()['instance_id'].to_dict()
        df2['user_cnt1'] = df2['user_id'].apply(lambda x: user_cnt.get(x, 0))
        df2['item_cnt1'] = df2['item_id'].apply(lambda x: item_cnt.get(x, 0))
        df2['shop_cnt1'] = df2['shop_id'].apply(lambda x: shop_cnt.get(x, 0))
        df2 = df2[['user_cnt1', 'item_cnt1', 'shop_cnt1', 'instance_id']]
        if d == 19:
            Df2 = df2
        else:
            Df2 = pd.concat([df2, Df2])
    data = pd.merge(data, Df2, on=['instance_id'], how='left')
    print('当前日期之前的cnt')
    for d in range(19, 26):
        # 19到25，25是test
        df1 = data[data['day'] < d]
        df2 = data[data['day'] == d]
        user_cnt = df1.groupby(by='user_id').count()['instance_id'].to_dict()
        item_cnt = df1.groupby(by='item_id').count()['instance_id'].to_dict()
        shop_cnt = df1.groupby(by='shop_id').count()['instance_id'].to_dict()
        df2['user_cntx'] = df2['user_id'].apply(lambda x: user_cnt.get(x, 0))
        df2['item_cntx'] = df2['item_id'].apply(lambda x: item_cnt.get(x, 0))
        df2['shop_cntx'] = df2['shop_id'].apply(lambda x: shop_cnt.get(x, 0))
        df2 = df2[['user_cntx', 'item_cntx', 'shop_cntx', 'instance_id']]
        if d == 19:
            Df2 = df2
        else:
            Df2 = pd.concat([df2, Df2])
    data = pd.merge(data, Df2, on=['instance_id'], how='left')

    print("前一个小时的统计量")

    return data

In [18]:
def convert_data(data):
    data['time'] = data.context_timestamp.apply(time2cov)
    data['day'] = data.time.apply(lambda x: int(x[8:10]))
    data['hour'] = data.time.apply(lambda x: int(x[11:13]))
    user_query_day = data.groupby(['user_id', 'day']).size(
    ).reset_index().rename(columns={0: 'user_query_day'})
    data = pd.merge(data, user_query_day, 'left', on=['user_id', 'day'])
    user_query_day_hour = data.groupby(['user_id', 'day', 'hour']).size().reset_index().rename(
        columns={0: 'user_query_day_hour'})
    data = pd.merge(data, user_query_day_hour, 'left',
                    on=['user_id', 'day', 'hour'])

    return data

In [19]:
def dataClean(raw_data):
    print("###############  Cleaning & Encoding categorical data to one-hot expression   ################")
    le = LabelEncoder()
    print("###############  item  ###############")
    for i in range(3):
        split = udf(lambda x: x.split(";")[i] if len(x.split(";")) > i else " ", StringType())
        raw_data = raw_data.select('*', split(raw_data['item_category_list']).alias('category_%d' % (i)))
    raw_data = raw_data.drop('item_category_list')
   
    for i in range(3):
        raw_data = raw_data.select('*', split(raw_data['item_property_list']).alias('property_%d' % (i)))
    raw_data = raw_data.drop('item_property_list')
    
    for col in ['item_id', 'item_brand_id', 'item_city_id']:
        string = udf(lambda s: s, StringType())
        raw_data.select(string(col))
        
    time_tran = udf(lambda t: time2cov(t))
    #raw_data['realtime'] = raw_data['context_timestamp'].apply(time2cov)
    raw_data = raw_data.select('*', time_tran(raw_data['context_timestamp']).alias('realtime'))
    
    for i in range(3):
        split_1 = udf(lambda y: y.split(";")[i].split(":")[0] if len(y.split(";")) > i else " ", StringType())
        raw_data = raw_data.select('*', split_1(raw_data['predict_category_property']).alias('predict_category_%d' % (i)))
    #raw_data.printSchema()
    
    print("###############  user  ###############")
    for col in ['user_id']:
        #raw_data[col] = le.fit_transform(raw_data[col])
        encode = udf(lambda l: le.fit_transform(l))
        raw_data = raw_data.select('*', encode(raw_data[col]))
    print('user 0,1 feature')
    
    mapping_g = udf(lambda g: 1 if g == -1 else 2)
    mapping_a = udf(lambda a: 1 if a == 1004 | a == 1005 | a == 1006 | a == 1007  else 2)
    mapping_o = udf(lambda o: 1 if o == -1 | o == 2003  else 2)
    mapping_s = udf(lambda s: 1 if s == -1 | s == 3000 | s == 3001  else 2)
    raw_data = raw_data.select('*', mapping_g(raw_data['user_gender_id']).alias('gender0'))
    raw_data = raw_data.select('*', mapping_a(raw_data['user_age_level']).alias('age0'))
    raw_data = raw_data.select('*', mapping_o(raw_data['user_occupation_id']).alias('occupation0'))
    raw_data = raw_data.select('*', mapping_s(raw_data['user_star_level']).alias('star0'))
    #raw_data.printSchema()
    
    print("###############  context  ###############")
    time_convert = udf(lambda t: pd.to_datetime(t))
    raw_data = raw_data.select('*', time_convert(raw_data['realtime']))
    raw_data.printSchema()
    mapping_l = udf(lambda l: len(str(l).split(';')))
    raw_data = raw_data.select('*', mapping_l(raw_data['predict_category_property']).alias('len_predict_category_property'))
                    
    print('context 0,1 feature')
    mapping_c = udf(lambda c: 1 if c == 4001 | c == 4002 | c == 4003 | c == 4004 | c == 4007  else 2)
    raw_data = raw_data.select('*', mapping_c(raw_data['context_page_id']).alias('context_page0'))
    

    print("###############  shop  ###############")
    mapping_shop = udf(lambda s: 0 if s <= 0.98 and s >= 0.96  else 1)
    for col in ['shop_id']:
        raw_data = raw_data.select('*', encode(raw_data[col]))
    
    raw_data = raw_data.select('*', mapping_shop(raw_data['shop_score_delivery']).alias('shop_score_delivery0'))
    raw_data.printSchema()
    
    return raw_data

In [30]:
#归一化(价格与最低价差值占总价格的百分比)
def price_normalization(df):
    #df['item_norm_price_score'] = (df['item_price_level'] - df['item_price_level'].min()) / df['item_price_level'].mean()
    df.describe('item_price_level').show()
    #df1.select('min(item_price_level)')
    #df = df.select('*',((df['item_price_level']-df1['min(item_price_level)'])/df1['avg(item_price_level)']).alias('item_norm_price_score'))
    
    
    #df = df.withColumn("min", df1['min(item_price_level)'])
    
    
    
    #df1.select('min(item_price_level)', 'avg(item_price_level)').show()
    #df = df.drop('item_price_level')
    return df

In [31]:
def featureExtraction(raw_data):
    print("###############  Extracting Training Features   ################")
    #user action series
    raw_data = raw_data.orderBy('realtime')
    print raw_data.columns
    # 商品的价格在同类商品中的比值
    print('一个item在同类商品中item_price_level的归一化分数…(在同类商品价格中的分布)(查询词的预测cat0_1_2相同 if( != ' '))')
    
    raw_data_g = raw_data.groupby('predict_category_0', 'predict_category_1', 'predict_category_2').count().alias("counts")
    raw_data_g = raw_data_g.drop('counts')
    raw_data_joined = raw_data_g.join(raw_data, ['predict_category_0', 'predict_category_1', 'predict_category_2'])
    item_norm_price_score = price_normalization(raw_data_joined)
    #raw_data_g = raw_data.groupby('predict_category_0', 'predict_category_1', 'predict_category_2').agg(min('item_price_level'), avg('item_price_level')).na.fill(0)
    #raw_data_g.select('min(item_price_level)', 'avg(item_price_level)')
    print(len(item_norm_price_score))
    print(len(raw_data))
    #raw_data['item_norm_price_score'] = item_norm_price_score['item_norm_price_score']
    raw_data = raw_data.join(item_norm_price_score)
    raw_data.printSchema()
    print('一个user有多少item_id,item_brand_id……')
    user_cnt = raw_data.groupby(['user_id'], as_index=False)['instance_id'].agg({'user_cnt': 'count'})
    raw_data = pd.merge(raw_data, user_cnt, on=['user_id'], how='left')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = raw_data.groupby([col, 'user_id'], as_index=False)['instance_id'].agg({str(col)+'_user_cnt': 'count'})
        raw_data = pd.merge(raw_data, item_shop_cnt, on=[col, 'user_id'], how='left')
        raw_data[str(col) + '_user_prob'] = raw_data[str(col) + '_user_cnt'] / raw_data['user_cnt']
    del raw_data['user_cnt']

    print('一个shop有多少item_id,item_brand_id,item_city_id,item_price_level……')
    itemcnt = raw_data.groupby(['shop_id'], as_index=False)['instance_id'].agg({'shop_cnt': 'count'})
    raw_data = pd.merge(raw_data, itemcnt, on=['shop_id'], how='left')
    for col in ['item_id',
                'item_brand_id','item_city_id','item_price_level',
                'item_sales_level','item_collected_level','item_pv_level']:
        item_shop_cnt = raw_data.groupby([col, 'shop_id'], as_index=False)['instance_id'].agg({str(col)+'_shop_cnt': 'count'})
        raw_data = pd.merge(raw_data, item_shop_cnt, on=[col, 'shop_id'], how='left')
        raw_data[str(col) + '_shop_prob'] = raw_data[str(col) + '_shop_cnt'] / raw_data['shop_cnt']
    del raw_data['shop_cnt']


    print('一个brand的平均价格指数（平均值，方差，衡量品牌的类型）…')

    brand_price_mean = raw_data.groupby('item_brand_id', as_index=False)['item_price_level'].agg({'brand_price_mean': 'mean'})
    brand_price_std = raw_data.groupby('item_brand_id', as_index=False)['item_price_level'].agg({'brand_price_std': 'std'})
    raw_data = pd.merge(raw_data, brand_price_mean, on=['item_brand_id'], how='left')
    raw_data = pd.merge(raw_data, brand_price_std, on=['item_brand_id'], how='left')


    print("###############  Constructing user_items_shop feature series  ################")
    # 用每个用户的浏览（购买）序列来编码 每个商品的embedding vector，以及每个店铺的embedding vector
    # 1. 构造每个用户的购买序列data frame (过滤掉浏览记录过少的用户)
    user_list = raw_data['user_id'].unique()
    item_list = raw_data['item_id'].unique()

    seq_item_id = [[] for i in range(len(user_list))]
    seq_item_brand_id = [[] for i in range(len(user_list))]
    seq_shop_id = [[] for i in range(len(user_list))]

    #seq_item_category_0 = [[] for i in range(len(user_list))]
    seq_item_category_1 = [[] for i in range(len(user_list))]
    #seq_item_category_2 = [[] for i in range(len(user_list))]

    seq_item_property_0 = [[] for i in range(len(user_list))]
    seq_item_property_1 = [[] for i in range(len(user_list))]
    seq_item_property_2 = [[] for i in range(len(user_list))]


    for i in range(len(user_list)):
        user_instance_series = raw_data.loc[raw_data['user_id'] == user_list[i]]
        seq_item_id[i] = user_instance_series['item_id'].tolist()
        seq_item_brand_id[i] = user_instance_series['item_brand_id'].tolist()
        #seq_shop_id[i] = user_instance_series['shop_id'].tolist()
        #seq_item_category_0[i] = user_instance_series['category_0'].tolist()
        seq_item_category_1[i] = user_instance_series['category_1'].tolist()
        #seq_item_category_2[i] = user_instance_series['category_2'].tolist()

        seq_item_property_0[i] = user_instance_series['property_0'].tolist()
        seq_item_property_1[i] = user_instance_series['property_1'].tolist()
        seq_item_property_2[i] = user_instance_series['property_2'].tolist()
        if(i%5000 == 0):
            print('%.2f%%' % (i/len(user_list) * 100))


    #train the embedding layer : output item2vec dense features
    print("###############  Training the Embedding vector of user items series  ################")
    spare_features = {'item_id','item_brand_id','category_1','property_0','property_1','property_2'}
    seq_data = {'item_id':seq_item_id,'item_brand_id':seq_item_brand_id,'category_1':seq_item_category_1,'property_0':seq_item_property_0,'property_1':seq_item_property_1,'property_2':seq_item_property_2}
    for feat in spare_features:
        if(feat == "category_1"):
            # raw_data[feat + '_vec'] = item2vecTraining2(feat, seq_data, raw_data[feat].unique(), size=5, window=3, min_count=1, workers=1, iter=3, sample=1e-4,negative=20)
            raw_data=raw_data.merge(item2vecTraining2(feat, seq_data, raw_data[feat].unique(), size=5, window=3, min_count=1, workers=1, iter=5, sample=1e-4,negative=20), left_on=feat,right_index=True,how='left')
#iter=3 min_count=1
        else:
            # raw_data[feat + '_vec'] = item2vecTraining2(feat, seq_data, raw_data[feat].unique(), size=50, window=3, min_count=1, workers=1, iter=3, sample=1e-4,negative=20)
            raw_data=raw_data.merge(item2vecTraining2(feat, seq_data, raw_data[feat].unique(), size=50, window=3, min_count=1, workers=1, iter=8, sample=1e-4,negative=20), left_on=feat,right_index=True,how='left')
    return raw_data


In [32]:
def item2vecTraining(feat,seq_data,item_list,**params):
    model = Word2Vec(seq_data[feat], **params)
    # 训练skip-gram模型; 默认window=5
    weights = model.wv.syn0
    vocab = dict([(k, v.index) for k, v in model.wv.vocab.items()])
    np.save("./embedded_weight/"+feat+".npy", weights)
    np.save("./embedded_weight/vocab_map/"+feat+"_map.npy",vocab)
    del model

In [33]:
def item2vecTraining2(feat,seq_data,key_list,**params):
    model = Word2Vec(seq_data[feat], **params)
    # 训练skip-gram模型; 默认window=5
    item_vec_dict = {k : model[k] for k in key_list}
    item_vec_series = pd.Series(item_vec_dict)
    print(item_vec_series)
    #del model
    return item_vec_series.to_frame(name=feat + '_vec')

In [34]:
def readData():
    # Load a text file and convert each line to a Row.
    train_data = sc.textFile("./data/train.txt")
    test_data = sc.textFile("./data/test.txt")
    train_pairs = train_data.map(lambda m: tuple(m.split(' '))).filter(lambda x:len(x)==27)
    train_pairs.cache()
    test_pairs = test_data.map(lambda m: tuple(m.split(' ')))
    test_pairs.cache()
    # Create the DataFrame from an RDD of tuples
    train_header = ['instance_id', 'item_id', 'item_category_list', 'item_property_list', 'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id', 'context_timestamp', 'context_page_id', 'predict_category_property', 'shop_id', 'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level', u'shop_score_service', 'shop_score_delivery', 'shop_score_description', 'is_trade']
    test_header = test_pairs.collect()[0]
    train_df = spark.createDataFrame(train_pairs, train_header)
    test_df = spark.createDataFrame(test_pairs, test_header)
    train_df = train_df.filter("instance_id!='instance_id'")  
    test_df = test_df.filter("instance_id!='instance_id'")  
    train_data = dataClean(train_df)
    test_data = dataClean(test_df)
    
    #all_data = featureExtraction(pd.concat([train_data, test_data]).reset_index(drop=True))
    all_data = train_data.union(test_data.withColumn("is_trade", lit('null')))
    all_data = featureExtraction(all_data)
    
    
    #print(type(all_data['item_id'].unique()))

    #np.save("./data/all_data.npy",all_data)
    #all_data.to_pickle('./data/all_data.pkl')
    # print(train_data.groupby(['predict_category_0', 'predict_category_1','predict_category_2'], as_index=False)['item_price_level'].apply(price_normalization))
    # print(all_data.groupby(['predict_category_0', 'predict_category_1', 'predict_category_2'], as_index=False)[
    #               'item_price_level'].apply(price_normalization))
    # print(sum(train_data.groupby(['predict_category_0', 'predict_category_1','predict_category_2'], as_index=False)['item_price_level'].apply(price_normalization) != 0))
    # print(train_data)

    return  train_data, test_data
    

In [35]:
readData()

###############  Cleaning & Encoding categorical data to one-hot expression   ################
###############  item  ###############
###############  user  ###############
user 0,1 feature
###############  context  ###############
root
 |-- instance_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- item_brand_id: string (nullable = true)
 |-- item_city_id: string (nullable = true)
 |-- item_price_level: string (nullable = true)
 |-- item_sales_level: string (nullable = true)
 |-- item_collected_level: string (nullable = true)
 |-- item_pv_level: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_gender_id: string (nullable = true)
 |-- user_age_level: string (nullable = true)
 |-- user_occupation_id: string (nullable = true)
 |-- user_star_level: string (nullable = true)
 |-- context_id: string (nullable = true)
 |-- context_timestamp: string (nullable = true)
 |-- context_page_id: string (nullable = true)
 |-- predict_category_property: s

Py4JJavaError: An error occurred while calling o3566.describe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 18 in stage 13.0 failed 1 times, most recent failure: Lost task 18.0 in stage 13.0 (TID 27, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/vchim/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/home/vchim/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/vchim/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 104, in <lambda>
    func = lambda _, it: map(mapper, it)
  File "<string>", line 1, in <lambda>
  File "/home/vchim/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 71, in <lambda>
    return lambda *a: f(*a)
  File "<ipython-input-19-e10f23bdaf6